# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.0.0
['tmp', 'weights', '.ipynb_checkpoints', 'models', '.jovianrc', 'model_name', 'Save, Load and Export Keras Models.ipynb']


## Task 2: Create Model

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['acc']
    )
    
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

## Task 4: Model Checkpoint During Training

In [4]:
checkpoint_dir = 'weights/'

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}'),
            monitor='val_acc', 
            save_weights_only=True, 
            save_best_only=True
        )
    ]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 1s 23us/sample - loss: 0.6900 - acc: 0.7645 - val_loss: 0.5193 - val_acc: 0.8113
Epoch 2/2
60000/60000 [==============================] - 1s 14us/sample - loss: 0.4327 - acc: 0.8493 - val_loss: 0.4345 - val_acc: 0.8491


In [5]:
os.listdir(checkpoint_dir)

['epoch_01.index',
 'checkpoint',
 'epoch_01.data-00000-of-00001',
 'epoch_02.data-00000-of-00001',
 'epoch_02.index']

## Task 5: Load Weights

In [6]:
# Random model
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.390696295166016, 0.054]


In [7]:
# Pretrained model
model.load_weights('weights/epoch_02')
print(model.evaluate(x_test, y_test, verbose=False))

[0.43453370032310484, 0.8491]


## Task 6: Saving Complete Model During Training

In [8]:
models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}.h5'),
            monitor='val_acc', 
            save_weights_only=False, 
            save_best_only=False
        )
    ]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 1s 21us/sample - loss: 0.6863 - acc: 0.7714 - val_loss: 0.4865 - val_acc: 0.8309
Epoch 2/2
60000/60000 [==============================] - 1s 14us/sample - loss: 0.4274 - acc: 0.8498 - val_loss: 0.4340 - val_acc: 0.8464


In [9]:
os.listdir(models_dir)

['epoch_01_acc_0.8241.h5',
 'epoch_02_acc_0.8360.h5',
 'epoch_01_acc_0.8164.h5',
 'epoch_02.h5',
 'epoch_01.h5',
 'epoch_02_acc_0.8435.h5']

## Task 7: Load Models

In [10]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.359277976989746, 0.1136]


In [12]:
model = tf.keras.models.load_model('models/epoch_02.h5')
model.summary()
print(model.evaluate(x_test, y_test, verbose=False))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
[0.4339824601173401, 0.8464]


## Task 8: Manually Saving Weights and Models

In [13]:
model.save_weights('tmp/manually_saved')
print(os.listdir('tmp'))

['manually_saved.index', 'checkpoint', 'manually_saved.data-00000-of-00001']


In [14]:
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['manually_saved.index', 'checkpoint', 'manually_saved.data-00000-of-00001', 'manually_saved_model.h5']


## Task 9: Exporting and Restoring Saved Model Format

In [15]:
model.save('model_name')
print(os.listdir('model_name'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_name/assets
['assets', 'saved_model.pb', 'variables']


In [16]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4339824549198151, 0.8464]
